Apprentissage de données $\phi$-FEM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

seed = 2023
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.experimental.numpy.random.seed(seed)
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
# Set a fixed value for the hash seed
os.environ["PYTHONHASHSEED"] = str(seed)
import dolfin as df
import time
from utils import *
from utils_training import *

import seaborn as sns

sns.set_theme()
sns.set_context("paper")
sns.set(rc={"xtick.bottom": True, "ytick.left": True})
colors = sns.color_palette("mako").as_hex()
my_cmap = sns.color_palette("viridis", as_cmap=True)


# Run

In [ ]:
pad_mode = [0, 1]  # 0 : constant, 1 : Reflect
padding_size = [0.025, 0.05, 0.1]
params = np.array(np.meshgrid(pad_mode, padding_size)).T.reshape(-1, 2)
params_tmp = np.zeros((np.shape(params)[0] + 1, np.shape(params)[1]))
params_tmp[1:, :] = params
params = params_tmp
reduced_params = params[:, :]
params = reduced_params
np.save("./params.npy", params)
params = np.load("./params.npy")
print(params)


In [ ]:
small_data = False
data = DataLoader(small_data)
level = 2

In [ ]:
index = 0
for param in params:
    mode, pad_size = param
    print(f"{index}/{len(params)}  {(mode, pad_size)=} ")
    if mode == 0:
        mode = "CONSTANT"
    elif mode == 1:
        mode = "REFLECT"
    agent = Agent(
        data,
        small_data,
        width=20,
        modes=10,
        nb_layers=4,
        pad_prop=pad_size,
        pad_mode=mode,
        index_test_case=index,
        loss_level=2,
    )
    epochs = 202
    ti0 = time.time()
    agent.train(epochs)
    duration = time.time() - ti0
    print("average time per epoch :", duration / epochs)
    index += 1
